In [172]:
import pyshtools as pysh
import numpy as np
from numpy import pi as pi

import RFmod as RF
import SLmod as SL
import xarray as xr

In [150]:
## Preliminaries

L = 63
lmax=10
Q = RF.sobolev_covariance(L, s=2, mu=0.2)
rhow = SL.rhow

# read in the present day sea level and ice thickness
sl0,ice0 = SL.get_sl_ice_data(L)
C = SL.ocean_function(sl0,ice0)

In [152]:
## For the input to our function, we would like a vector formed by reshaping a gridded direct load, zeta

zeta_glq = -rhow*RF.random_field(Q)
zeta_gr = -rhow*RF.random_field(Q).to_array()
zeta = zeta_gr.reshape(-1)

In [147]:
## Version of the function which just uses a normal GLQRealGrid as input

def potential0(zeta_glq, L, lmax=10):

    ## Solve the sea level equation
    _,_,solution,_,_ = SL.fingerprint(C, zeta_glq)

    ## Expand the solution in spherical harmonics. Clm = solution[0,l,m] and Slm = solution[1,l,m]
    coeffs = pysh.expand.SHExpandGLQ(solution.to_array(),lmax_calc=lmax)

    ## Populate a vector (1-D np array) with the expanded coefficients, in ascending order of l (2<=l<=lmax), doing Clm in reverse order then Slm in standard order for each l
    coeffs_vec = np.zeros(((lmax+1)**2)-4)
    
    for l in range(2,lmax+1):
        coeffs_vec[((l)**2)-4:((l+1)**2)-4] = np.concatenate((coeffs[0,l,0:l+1][::-1],coeffs[1,l,1:l+1]))
    
    return coeffs_vec

In [162]:
## Full version of the function

def potential(zeta, L, lmax=10):

    ## Convert zeta to a GLQGrid
    zeta_glq = pysh.SHGrid.from_array(zeta.reshape(L+1,2*(L+1)),grid='GLQ')

    ## Solve the sea level equation
    _,_,solution,_,_ = SL.fingerprint(C, zeta_glq)

    ## Expand the solution in spherical harmonics. Clm = solution[0,l,m] and Slm = solution[1,l,m]
    coeffs = pysh.expand.SHExpandGLQ(solution.to_array(),lmax_calc=lmax)

    ## Populate a vector (1-D np array) with the expanded coefficients, in ascending order of l (2<=l<=lmax), doing Clm in reverse order then Slm in standard order for each l
    coeffs_vec = np.zeros(((lmax+1)**2)-4)
    
    for l in range(2,lmax+1):
        coeffs_vec[((l)**2)-4:((l+1)**2)-4] = np.concatenate((coeffs[0,l,0:l+1][::-1],coeffs[1,l,1:l+1]))
    
    return coeffs_vec


In [170]:
## Vague attempt at a class

import os
import sys

class PotentialSolver:
    def __init__(self, L, lmax=10):
        self.L = L
        self.lmax = lmax

    def A(self, zeta, suppress_output=True):
        zeta_glq = pysh.SHGrid.from_array(zeta.reshape(self.L+1,2*(self.L+1)),grid='GLQ')

        # Suppress output if requested
        if suppress_output:
            original_stdout = sys.stdout  # Save a reference to the original standard output
            sys.stdout = open(os.devnull, 'w')

        _,_,solution,_,_ = SL.fingerprint(C, zeta_glq)

        # Restore standard output if it was suppressed
        if suppress_output:
            sys.stdout.close()
            sys.stdout = original_stdout

        coeffs = pysh.expand.SHExpandGLQ(solution.to_array(),lmax_calc=self.lmax)
        coeffs_vec = np.zeros(((self.lmax+1)**2)-4)
    
        for l in range(2,self.lmax+1):
            coeffs_vec[((l)**2)-4:((l+1)**2)-4] = np.concatenate((coeffs[0,l,0:l+1][::-1],coeffs[1,l,1:l+1]))
        
        return coeffs_vec

In [173]:
## Testing the class

solve = PotentialSolver(L, lmax=10)
solve.A(zeta)

array([ 0.19456119, -0.06530445, -0.0524054 ,  0.0937309 ,  0.1513918 ,
       -0.05741823, -0.02622757,  0.10938095, -0.08580362, -0.02373222,
        0.16795801, -0.20035653, -0.01065972, -0.04445102, -0.02883192,
       -0.06176121, -0.02891048,  0.01480976,  0.07603586, -0.01299593,
        0.0267921 , -0.01998635,  0.01231964,  0.02002232, -0.03838759,
       -0.04724378, -0.00127344,  0.00426204, -0.01095402, -0.01002708,
       -0.03123467,  0.08220392,  0.0046136 , -0.01907973,  0.00422204,
        0.02738494, -0.04034152, -0.04091192,  0.02084734,  0.03872135,
        0.00384089,  0.00542826,  0.00126144, -0.00164106, -0.02799588,
       -0.01904044, -0.00530083, -0.01061337,  0.02242847,  0.063668  ,
        0.01783849, -0.03498052, -0.01057171, -0.02159634,  0.01436461,
       -0.00557634,  0.02235869,  0.02380941, -0.04353325, -0.00228823,
       -0.03164915,  0.00997883,  0.00628253, -0.00632775, -0.01026039,
       -0.01093916, -0.02205068,  0.00110483, -0.00060758,  0.01